In [1]:
import arviz as az
import pymc4 as pm
import math
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import time

from tqdm import tqdm
from pymc4.distributions.transforms import BackwardTransform

tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
class Interval(BackwardTransform):
    name = "interval"

    def __init__(self, lower_limit, upper_limit):
        transform = tf.cond(
            tf.math.is_inf(lower_limit),
            lambda: tf.cond(
                tf.math.is_inf(upper_limit),
                lambda: tfb.Identity(),
                lambda: tfb.Chain([tfb.Shift(upper_limit), tfb.Scale(-1), tfb.Exp()]),  # upper - exp(x)
            ),
            lambda: tf.cond(
                tf.math.is_inf(upper_limit),
                lambda: tfb.Chain([tfb.Shift(lower_limit), tfb.Exp()]),  # exp(x) + lower
                lambda: tfb.Sigmoid(low=lower_limit, high=upper_limit),  # interval
            ),
        )
        super().__init__(transform)

In [3]:
experiments = np.array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

@pm.model
def model():
    prob = yield pm.Uniform('p', 0., 1., transform=Interval(0., 1.))
    ll = yield pm.Bernoulli('ll', prob, observed=experiments, transform=Interval(0., 1.))

In [4]:
advi = pm.fit(model())

TypeError: To be compatible with tf.eager.defun, Python functions must return zero or more Tensors; in compilation of <function Interval.__init__.<locals>.<lambda>.<locals>.<lambda> at 0x13a41ff80>, found return value of type <class 'tensorflow_probability.python.bijectors.identity.Identity'>, which is not a Tensor.